In [ ]:
!pip install --upgrade pip
!pip install "snowflake-connector-python[pandas]" "snowflake-snowpark-python[pandas]" snowflake-snowpark-python==1.9.0 fosforio fosforml numpy pandas matplotlib scikit-learn xgboost seaborn python-dateutil tqdm holidays faker
!pip install --upgrade --q snowflake-snowpark-python==1.9.0
!pip uninstall urllib3 -y
!pip install urllib3==1.26.15
!pip install fosforml 

     |████████████████████████████████| 1.8MB 3.7MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 12.5 MB/s eta 0:00:00


In [1]:
from fosforio import snowflake
from fosforio import get_dataframe
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
#import seaborn as sns
#from imblearn.over_sampling import SMOTE
from scipy.stats.mstats import winsorize
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import joblib
from fosforml import *
from fosforml.constants import MLModelFlavours

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.


Matplotlib created a temporary cache directory at /tmp/matplotlib-l_tovetz because the default path (/home/mosaic-ai/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
/tmp/pip_packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


In [2]:
snowflake.get_connection(connection_name="HR Attrition conn")
df = get_dataframe("Final_HR_Attrition_Data")
df

Connection object created: <snowflake.connector.connection.SnowflakeConnection object at 0x7fe914c3f7f0>
Please close the connection after use!
Reading dataframe from snowflake native connector


,SALARY,SENIORITY,TENURE_MONTHS,MONTHS_AFTER_COLLEGE,BIRTH_YEAR,AGE,MAPPED_ROLE_CLEAN,SEX,ETHNICITY,HOSPITAL_TYPE,...,OVER_TIME_HOURS,CHURN,BUSINESS_TRAVEL,ENVIRONMENT_SATISFACTION,JOB_SATISFACTION,MARTIAL_STATUS,PERCENTAGE_SALARY_HIKE,PERFORMANCE_RATING,RELATIONSHIP_SATISFACTION,WORK_LIFE_BALANCE
0,68271.546,3,8,116,1984,40,occupational,F,Black,Childrens,...,15,False,Travel_Rarely,High,Low,Married,11,Excellent,Medium,Best
1,70471.235,3,37,18,1992,32,nurse,F,Hispanic,Psychiatric,...,10,True,Travel_Rarely,High,Medium,Married,11,Outstanding,Low,Good
2,53490.238,2,47,16,1988,36,emt,F,White,Psychiatric,...,14,False,Travel_Rarely,Very High,Low,Single,19,Good,Low,Bad
3,47749.212,1,40,32,1983,41,nurse,M,Hispanic,Acute Care Hospitals,...,3,False,Travel_Frequently,High,High,Single,13,Low,Medium,Bad
4,61379.954,1,29,11,1992,32,occupational,F,Hispanic,Psychiatric,...,1,True,Travel_Frequently,High,Medium,Married,19,Low,Medium,Bad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,69870.670,2,42,14,1995,29,nurse,M,Black,Acute Care - Department of Defense,...,6,False,Travel_Rarely,Medium,Very High,Divorced,13,Excellent,Low,Good
299996,79047.028,5,106,164,1975,49,technologist,M,Hispanic,Acute Care - Department of Defense,...,6,True,Travel_Rarely,Low,Medium,Divorced,14,Excellent,Medium,Best
299997,51505.375,1,12,69,1985,39,social,F,Multiple,Acute Care - Department of Defense,...,9,False,Travel_Rarely,Low,Medium,Married,18,Good,Very High,Best
299998,64988.218,2,9,76,1985,39,occupational,F,API,Childrens,...,14,False,Non-Travel,Very High,Very High,Single,15,Outstanding,Low,Better


In [3]:
print(df.isnull().sum())

SALARY                           0
SENIORITY                        0
TENURE_MONTHS                    0
MONTHS_AFTER_COLLEGE             0
BIRTH_YEAR                       0
AGE                              0
MAPPED_ROLE_CLEAN                0
SEX                              0
ETHNICITY                        0
HOSPITAL_TYPE                    0
HOSPITAL_OWNERSHIP               0
COMPANY_NAME                     0
CITY                             0
STATE                            0
DISTANCE                         0
DEGREE_CLEAN                     0
SCHOOL_END_DATE                  0
JOB_START_DATE                   0
JOB_END_DATE                 90102
USER_ID                          0
OVER_TIME_HOURS                  0
CHURN                            0
BUSINESS_TRAVEL                  0
ENVIRONMENT_SATISFACTION         0
JOB_SATISFACTION                 0
MARTIAL_STATUS                   0
PERCENTAGE_SALARY_HIKE           0
PERFORMANCE_RATING               0
RELATIONSHIP_SATISFA

In [4]:
df = df.dropna()

In [10]:
print(df.isnull().sum())

SALARY                       0
SENIORITY                    0
TENURE_MONTHS                0
MONTHS_AFTER_COLLEGE         0
BIRTH_YEAR                   0
AGE                          0
MAPPED_ROLE_CLEAN            0
SEX                          0
ETHNICITY                    0
HOSPITAL_TYPE                0
HOSPITAL_OWNERSHIP           0
COMPANY_NAME                 0
CITY                         0
STATE                        0
DISTANCE                     0
DEGREE_CLEAN                 0
OVER_TIME_HOURS              0
CHURN                        0
BUSINESS_TRAVEL              0
ENVIRONMENT_SATISFACTION     0
JOB_SATISFACTION             0
MARTIAL_STATUS               0
PERCENTAGE_SALARY_HIKE       0
PERFORMANCE_RATING           0
RELATIONSHIP_SATISFACTION    0
WORK_LIFE_BALANCE            0
dtype: int64


In [5]:
df = df.drop(["USER_ID", "JOB_START_DATE", "JOB_END_DATE", "SCHOOL_END_DATE"], axis = 1)

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
CATEGORICAL_COLUMNS = ["MAPPED_ROLE_CLEAN","SEX", "ETHNICITY","HOSPITAL_TYPE", "HOSPITAL_OWNERSHIP","COMPANY_NAME","CITY","STATE","DISTANCE", 
                       "DEGREE_CLEAN","BUSINESS_TRAVEL","ENVIRONMENT_SATISFACTION","JOB_SATISFACTION","MARTIAL_STATUS","PERFORMANCE_RATING","RELATIONSHIP_SATISFACTION","WORK_LIFE_BALANCE"]
NUMERICAL_COLUMNS = ["SALARY", "SENIORITY", "TENURE_MONTHS", "MONTHS_AFTER_COLLEGE", "BIRTH_YEAR","AGE", "OVER_TIME_HOURS", "PERCENTAGE_SALARY_HIKE",]
LABEL_COLUMNS = ["CHURN"]
OUTPUT_COLUMNS = ["PREDICTION"]

In [8]:
feature_columns = CATEGORICAL_COLUMNS + NUMERICAL_COLUMNS 

In [9]:
feature_columns = [col for col in feature_columns if col in df.columns]
LABEL_COLUMNS = [col for col in LABEL_COLUMNS if col in df.columns]

In [10]:
X = df[feature_columns]
Y = df[LABEL_COLUMNS]

In [11]:
X.head()

,MAPPED_ROLE_CLEAN,SEX,ETHNICITY,HOSPITAL_TYPE,HOSPITAL_OWNERSHIP,COMPANY_NAME,CITY,STATE,DISTANCE,DEGREE_CLEAN,...,RELATIONSHIP_SATISFACTION,WORK_LIFE_BALANCE,SALARY,SENIORITY,TENURE_MONTHS,MONTHS_AFTER_COLLEGE,BIRTH_YEAR,AGE,OVER_TIME_HOURS,PERCENTAGE_SALARY_HIKE
0,occupational,F,Black,Childrens,Department of Defense,Village Practice Management Co. LLC,Lewiston,ME,>10 Miles,Professional Certificate,...,Medium,Best,68271.546,3,8,116,1984,40,15,11
1,nurse,F,Hispanic,Psychiatric,Proprietary,McKesson Corp.,Charlottesville,VA,>10 Miles,Undergraduate Diploma,...,Low,Good,70471.235,3,37,18,1992,32,10,11
2,emt,F,White,Psychiatric,Government - Hospital District or Authority,24-7 Bright Star Healthcare LLC,Evansville,IN,2-5 Miles,Masters Degree,...,Low,Bad,53490.238,2,47,16,1988,36,14,19
3,nurse,M,Hispanic,Acute Care Hospitals,Government - Hospital District or Authority,AmerisourceBergen Corp.,Lakeland,FL,<2 Miles,Undergraduate Diploma,...,Medium,Bad,47749.212,1,40,32,1983,41,3,13
4,occupational,F,Hispanic,Psychiatric,Government - Federal,Cleveland Clinic,Buffalo,NY,5-10 Miles,Doctoral Degree,...,Medium,Bad,61379.954,1,29,11,1992,32,1,19


In [12]:
Y.head()

,CHURN
0,False
1,True
2,False
3,False
4,True


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size = 0.20, random_state = 42)

In [14]:
## train_df and test_df are both snowpark dataframes
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline

categorical_transformer = make_pipeline(
    SimpleImputer(strategy='constant', fill_value='missing'),
    OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
)

numerical_transformer = make_pipeline(
    SimpleImputer(strategy='mean'),
    MinMaxScaler(clip=True)
)

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, CATEGORICAL_COLUMNS),
        ('num', numerical_transformer, NUMERICAL_COLUMNS)
    ]
)


pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier())
])

pipeline.fit(X_train, y_train)

result = pipeline.predict(X_test)

In [15]:
result

array([0, 0, 1, ..., 1, 0, 1])

In [16]:
from joblib import dump, load
filename = "HR_Attrition_ml_model.joblib"
dump(pipeline, filename)

['HR_Attrition_ml_model.joblib']

In [17]:
import requests

In [18]:
test = X_test
train = X_train
train["CHURN"] = train["CHURN"].apply(lambda x: 1 if x else 0)
pred = model.predict(test)
pred["probabilities"] = model.predict_proba(test)[["predict_proba_True"]]
test["PREDICTION"] = pred["PREDICTION"]
test["probabilities"] = pred["probabilities"]
test["CHURN"] = test["CHURN"].apply(lambda x: 1 if x else 0)

y_prob = np.array(test["probabilities"])
y_test = pred["CHURN"]
y_pred = pred["PREDICTION"]
X_train = train.drop(["CHURN"], axis=1)
X_test = pred.drop(["CHURN", "PREDICTION", "probabilities"], axis=1)
y_train = train["CHURN"]

KeyError: 'CHURN'

In [19]:
y_pred = pipeline.predict (X_test)
y_prob = pipeline.predict_proba(X_test)

In [20]:
#@scoring_func
def score(model, request):
    payload_dict = request.json["payload"]
    data = pd.DataFrame(payload_dict,index=[0])
    prediction = str(model.predict(data)[0])
    return prediction

In [21]:
## registering the model in refract.
tmp = register_model(pipeline, 
               score, 
               name="HR_Analytic_ml_model", 
               description="HR_ANALYTICS_model_trained_using _ml",
               flavour=MLModelFlavours.sklearn,
               model_type="classification",
               init_script="\\n pip install fosforml \\n pip install fosforio[snowflake] \\n pip install seaborn \\n pip install snowflake-connector-python[pandas] \\n pip install joblib==1.3.2 scikit-learn=1.3.2",
               y_true=y_test,
               y_pred=y_pred, 
               features=X_train.columns,
               labels=[0,1],
               input_type="json", 
               explain_ai=True,
               prob=y_prob,
               x_train=X_train, 
               x_test=X_test, 
               y_train=y_train,
               y_test=y_test,
               feature_names=X_train.columns.tolist(),
               original_features=X_train.columns.tolist(),
               feature_ids=X_train.columns,
               target_names=['NOT LEFT','LEFT'],
               kyd=True, kyd_score = True)


Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


In [22]:
payload  = {"payload": X_test.iloc[0].to_dict()}
payload

{'payload': {'MAPPED_ROLE_CLEAN': 'technologist',
  'SEX': 'F',
  'ETHNICITY': 'Black',
  'HOSPITAL_TYPE': 'Acute Care - Department of Defense',
  'HOSPITAL_OWNERSHIP': 'Proprietary',
  'COMPANY_NAME': 'Encompass Health Corp.',
  'CITY': 'York',
  'STATE': 'PA',
  'DISTANCE': '>10 Miles',
  'DEGREE_CLEAN': 'Undergraduate Diploma',
  'BUSINESS_TRAVEL': 'Travel_Rarely',
  'ENVIRONMENT_SATISFACTION': 'Very High',
  'JOB_SATISFACTION': 'High',
  'MARTIAL_STATUS': 'Married',
  'PERFORMANCE_RATING': 'Good',
  'RELATIONSHIP_SATISFACTION': 'Medium',
  'WORK_LIFE_BALANCE': 'Better',
  'SALARY': 32715.087,
  'SENIORITY': 1,
  'TENURE_MONTHS': 5,
  'MONTHS_AFTER_COLLEGE': 20,
  'BIRTH_YEAR': 1994,
  'AGE': 30,
  'OVER_TIME_HOURS': 9,
  'PERCENTAGE_SALARY_HIKE': 21}}